In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/train.csv')

In [7]:
columnsToAvgPerPropID = ['prop_id','prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price']
for colname in columnsToAvgPerPropID:
    df[colname+"_mean"] = df.groupby('prop_id')[colname].mean()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,click_bool,gross_bookings_usd,booking_bool,prop_id_mean,prop_starrating_mean,prop_review_score_mean,prop_brand_bool_mean,prop_location_score1_mean,prop_location_score2_mean,prop_log_historical_price_mean
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,0,NaN,0,1.0,2.0,0.0,1.0,3.04,NaN,3.645246
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,0,NaN,0,2.0,0.0,4.0,1.0,0.69,0.036100,3.300000
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,0,NaN,0,3.0,3.0,3.5,1.0,0.69,0.035055,4.165122
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,0,NaN,0,4.0,5.0,4.5,1.0,4.88,0.115500,4.965000
5,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,30184,4,4.5,...,0,NaN,0,5.0,0.0,0.0,0.0,6.02,NaN,3.533333
6,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,44147,3,3.5,...,0,NaN,0,6.0,3.0,0.0,0.0,2.30,0.018763,3.664000
7,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,50984,2,0.0,...,0,NaN,0,7.0,0.0,5.0,0.0,6.49,0.124700,4.374286
8,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,53341,4,4.0,...,0,NaN,0,8.0,3.0,4.0,0.0,3.99,0.071900,3.924000
9,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,56880,4,4.0,...,0,NaN,0,9.0,2.0,3.0,1.0,1.79,0.042000,4.031765
